In [ ]:
!pip install tensorflow
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Model, Sequential # Importing Sequential here
from tensorflow.keras.layers import Input, LSTM, Bidirectional, GRU, Dense, Dropout, Attention
from tensorflow.keras.layers import Permute, Multiply, Lambda
import tensorflow.keras.backend as K

In [ ]:
# Download stock data
ticker = "BAJFINANCE.NS"
data = yf.download(ticker, start="2010-01-01", end="2025-01-01")

[*********************100%***********************]  1 of 1 completed


In [ ]:
# Feature Engineering
data['SMA_50'] = data['Close'].rolling(window=50).mean()
data['SMA_200'] = data['Close'].rolling(window=200).mean()
data['EMA_50'] = data['Close'].ewm(span=50, adjust=False).mean()
data['EMA_200'] = data['Close'].ewm(span=200, adjust=False).mean()

In [ ]:
# Relative Strength Index (RSI)
def compute_rsi(data, window=14):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / (loss + 1e-10)  # Prevent division by zero
    return 100 - (100 / (1 + rs))

data['RSI'] = compute_rsi(data)

In [ ]:

# MACD (Moving Average Convergence Divergence)
ema12 = data['Close'].ewm(span=12, adjust=False).mean()
ema26 = data['Close'].ewm(span=26, adjust=False).mean()
data['MACD'] = ema12 - ema26


In [ ]:
# Bollinger Bands
data['BB_Std'] = data['Close'].rolling(50).std()
data['BB_Upper'] = data['SMA_50'] + 2 * data['BB_Std']
data['BB_Lower'] = data['SMA_50'] - 2 * data['BB_Std']

In [ ]:
# Price Change Features
data['Daily Change'] = data['Close'].diff()
data['Weekly Change'] = data['Close'].diff(7)

In [ ]:
data.dropna(inplace=True)
features = ['Close', 'SMA_50', 'SMA_200', 'EMA_50', 'EMA_200', 'RSI', 'MACD', 'BB_Upper', 'BB_Lower', 'Daily Change', 'Weekly Change']
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data[features].values)

In [ ]:
# Prepare supervised learning data
def create_features_labels(data, time_steps=30):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i+time_steps])
        y.append(data[i+time_steps, 0])
    return np.array(X), np.array(y)

time_steps = 30
X, y = create_features_labels(scaled_data, time_steps)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [ ]:
# Reshape for BiLSTM
X_train_bilstm = X_train.reshape(X_train.shape[0], X_train.shape[1], len(features))
X_test_bilstm = X_test.reshape(X_test.shape[0], X_test.shape[1], len(features))

In [ ]:
# BiLSTM Model
bilstm_model = Sequential([
    Bidirectional(LSTM(100, return_sequences=True), input_shape=(time_steps, len(features))),
    Dropout(0.2),
    Bidirectional(LSTM(50)),
    Dropout(0.2),
    Dense(25),
    Dense(1)
])

bilstm_model.compile(optimizer='adam', loss='mean_squared_error')
bilstm_model.fit(X_train_bilstm, y_train, epochs=50, batch_size=32, validation_data=(X_test_bilstm, y_test))
bilstm_predictions = bilstm_model.predict(X_test_bilstm).flatten()

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


87/87 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 0.0881 - val_loss: 0.0076
Epoch 2/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0063 - val_loss: 0.0128
Epoch 3/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0053 - val_loss: 0.0041
Epoch 4/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0047 - val_loss: 0.0104
Epoch 5/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0042 - val_loss: 0.0072
Epoch 6/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0044 - val_loss: 0.0091
Epoch 7/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.0037 - val_loss: 0.0089
Epoch 8/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0038 - val_loss: 0.0051
Epoch 9/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0040 - val_loss: 0.0062
Epoch 10/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0034 - val_loss: 0.0057
Epoch 11/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0039 - val_loss: 0.0211
Epoch 12/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0037 - val_l

In [ ]:
# BiGRU Model
bigru_model = Sequential([
    Bidirectional(GRU(50, return_sequences=True), input_shape=(time_steps, X.shape[2])),
    Dropout(0.2),
    Bidirectional(GRU(50)),
    Dropout(0.2),
    Dense(25, activation='relu'),
    Dense(1)
])
bigru_model.compile(optimizer='adam', loss='mse')
bigru_model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)
bigru_predictions = bigru_model.predict(X_test)

Epoch 1/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.1420
Epoch 2/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0125
Epoch 3/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0087
Epoch 4/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0080
Epoch 5/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0071
Epoch 6/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0067
Epoch 7/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0055
Epoch 8/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0047
Epoch 9/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0041
Epoch 10/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0042
Epoch 11/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0045
Epoch 12/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0043
Epoch 13/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0043
Epoch 14/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0035
Epoch 15/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0037
Epoc

In [ ]:
# Accuracy Calculation
def calculate_accuracy(y_true, y_pred):
    mape = mean_absolute_percentage_error(y_true, y_pred) * 100
    return 100 - mape  # Accuracy in percentage

print("BiLSTM Accuracy:", calculate_accuracy(y_test, bilstm_predictions), "%")
print("BiGRU Accuracy:", calculate_accuracy(y_test, bigru_predictions), "%")

BiLSTM Accuracy: 97.01750544208434 %
BiGRU Accuracy: 97.13568743117087 %


In [ ]:
# Hybrid BiLSTM + BiGRU Model
hybrid_model = Sequential([
    Bidirectional(LSTM(50, return_sequences=True), input_shape=(time_steps, X.shape[2])),
    Dropout(0.2),
    Bidirectional(GRU(50, return_sequences=True)),
    Dropout(0.2),
    Bidirectional(LSTM(50)),
    Dropout(0.2),
    Dense(25, activation='relu'),
    Dense(1)
])
hybrid_model.compile(optimizer='adam', loss='mse')
hybrid_model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)
hybrid_predictions = hybrid_model.predict(X_test)

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


87/87 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - loss: 0.0813
Epoch 2/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 0.0070
Epoch 3/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.0061
Epoch 4/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0056
Epoch 5/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0048
Epoch 6/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0045
Epoch 7/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - loss: 0.0039
Epoch 8/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 0.0046
Epoch 9/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.0045
Epoch 10/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0032
Epoch 11/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0044
Epoch 12/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0034
Epoch 13/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0030
Epoch 14/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0036
Epoch 15/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0035
Epoch 16/50
87/

In [ ]:
# Accuracy Calculation
def calculate_accuracy(y_true, y_pred):
    mape = mean_absolute_percentage_error(y_true, y_pred) * 100
    return 100 - mape  # Accuracy in percentage

In [ ]:
print("Hybrid BiLSTM + BiGRU Accuracy:", calculate_accuracy(y_test, hybrid_predictions), "%")

Hybrid BiLSTM + BiGRU Accuracy: 96.04721629340845 %


In [ ]:
# Attention Layer
def attention_layer(inputs):
    attention = Dense(1, activation='tanh')(inputs)
    attention = Permute((2, 1))(attention)
    attention = Dense(inputs.shape[1], activation='softmax')(attention)
    attention = Permute((2, 1))(attention)
    attention = Multiply()([inputs, attention])
    return Lambda(lambda x: K.sum(x, axis=1))(attention)

In [ ]:

# Hybrid BiGRU + Attention Model
inputs = Input(shape=(time_steps, X_train.shape[2]))
x = Bidirectional(GRU(50, return_sequences=True))(inputs)
x = Dropout(0.2)(x)
x = attention_layer(x)
x = Dense(25, activation='relu')(x)
output = Dense(1)(x)

hybrid_model = Model(inputs, output)
hybrid_model.compile(optimizer='adam', loss='mse')
hybrid_model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)
hybrid_predictions = hybrid_model.predict(X_test)

Epoch 1/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.1531
Epoch 2/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0036
Epoch 3/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0025
Epoch 4/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0021
Epoch 5/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0018
Epoch 6/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0019
Epoch 7/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0013
Epoch 8/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0014
Epoch 9/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0019
Epoch 10/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0011
Epoch 11/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0011
Epoch 12/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 9.4434e-04
Epoch 13/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0011
Epoch 14/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0011
Epoch 15/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0011
Epoch 16/50


In [ ]:
print("Hybrid BiGRU + Attention Accuracy:", calculate_accuracy(y_test, hybrid_predictions), "%")

Hybrid BiGRU + Attention Accuracy: 96.22449036291606 %


In [ ]:
# BiLSTM + Attention Model
input_shape = (X_train.shape[1], X_train.shape[2])
inputs = Input(shape=input_shape)
x = Bidirectional(LSTM(100, return_sequences=True))(inputs)
x = Dropout(0.5)(x)
x = Bidirectional(LSTM(50, return_sequences=True))(x)
x = Dropout(0.5)(x)

x = attention_layer(x)
x = Dense(25, activation='relu')(x)
outputs = Dense(1)(x)

hybrid_model = Model(inputs, outputs)
hybrid_model.compile(optimizer='adam', loss='mean_squared_error')


In [ ]:
# Training
history = hybrid_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Prediction
hybrid_predictions = hybrid_model.predict(X_test).flatten()

Epoch 1/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - loss: 0.1099 - val_loss: 0.0660
Epoch 2/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0037 - val_loss: 0.0200
Epoch 3/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0027 - val_loss: 0.0133
Epoch 4/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0025 - val_loss: 0.0345
Epoch 5/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0024 - val_loss: 0.0108
Epoch 6/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0024 - val_loss: 0.0093
Epoch 7/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0022 - val_loss: 0.0086
Epoch 8/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0023 - val_loss: 0.0363
Epoch 9/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0017 - val_loss: 0.0206
Epoch 10/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 0.0018 - val_loss: 0.0042
Epoch 11/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0021 - val_loss: 0.0496
Epoch 12/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0

In [ ]:
# Accuracy Calculation
def calculate_accuracy(y_true, y_pred):
    mape = mean_absolute_percentage_error(y_true, y_pred) * 100
    return 100 - mape

print("BiLSTM + Attention Accuracy:", calculate_accuracy(y_test, hybrid_predictions), "%")

BiLSTM + Attention Accuracy: 94.305661637462 %
